In [32]:
# Importing the contract addresses file fetched from ethereum etl

import pandas as pd 

df = pd.read_csv('./contracts_ERC20.csv')

df.head()

,address,bytecode,function_sighashes,is_erc20,is_erc721,block_timestamp,block_number,block_hash
0,0x0ef97d3e342b29c18bcbcb0426e3b5de4879c53c,0x6060604052361561008d5760e060020a600035046306...,"[0x06fdde03,0x095ea7b3,0x18160ddd,0x23b872dd,0...",True,False,2017-01-07T17:16:15Z,2953240,0x2b0dca05c9c8a922018ccf54cb522b9e2a715768d0da...
1,0xc2eb8dc6ce9c28e9a34d91da242e87c1d67b599b,0x6080604052600436106100c5576000357c0100000000...,"[0x06fdde03,0x095ea7b3,0x18160ddd,0x23b872dd,0...",True,False,2020-01-01T10:33:46Z,9195381,0x9edf9b6e92fa07370320c424a1ad5cb56458bdd15c58...
2,0xe002d045a781c8162e0aa3fcdb280dc4b05df187,0x6080604052600436106100a4576000357c0100000000...,"[0x06fdde03,0x095ea7b3,0x18160ddd,0x23b872dd,0...",True,False,2020-01-01T13:17:02Z,9195970,0x728b32226eec7cdfe5e3509f06fcfd9ea8b0d0717f61...
3,0x02585e4a14da274d02df09b222d4606b10a4e940,0x606060405260043610610107576000357c0100000000...,"[0x05d2035b,0x06fdde03,0x095ea7b3,0x18160ddd,0...",True,False,2017-11-09T09:57:52Z,4519171,0xcbe3a0875480b3edd237ce35eae8f77abe3564b1fdb9...
4,0xc814d43adfcb1e466e07eb0089e0580cfadd8d35,0x606060405236156100e35763ffffffff7c0100000000...,"[0x05d2035b,0x06fdde03,0x095ea7b3,0x18160ddd,0...",True,False,2017-11-09T14:19:17Z,4520332,0x3185ee8a09a9f2d7feb655bf69561ada589ddbf14db1...


In [37]:
#fetching contract abi-json from etherscan api

import requests

registry = pd.DataFrame(columns=['address','abi-json'])

def get_contract_info(contract_address):
    url = 'https://api.etherscan.io/api?module=contract&action=getabi&address={}&apikey=MEYEGN5KWUNXBGCJ1NTZ663M53MTETSU1E'.format(contract_address)
    response = requests.get(url)
    return response.json()


for i in range(len(df)):
    contract_address = df.iloc[i]['address']
    contract_abi = get_contract_info(contract_address)
    registry = registry.append({'address':contract_address,'abi-json':contract_abi},ignore_index=True)



registry.head()

/var/folders/b6/sm3y1rf13_128cwdjvcwlcdr0000gn/T/ipykernel_60019/2695721402.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  registry = registry.append({'address':contract_address,'abi-json':contract_abi},ignore_index=True)
/var/folders/b6/sm3y1rf13_128cwdjvcwlcdr0000gn/T/ipykernel_60019/2695721402.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  registry = registry.append({'address':contract_address,'abi-json':contract_abi},ignore_index=True)
/var/folders/b6/sm3y1rf13_128cwdjvcwlcdr0000gn/T/ipykernel_60019/2695721402.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  registry = registry.append({'address':contract_address,'abi-json':contract_abi},ignore_index=True)
/var/folders/b6/sm3y1rf13_128cwdjvcwlcdr0000gn/T/

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [41]:
len(registry)

677

In [42]:
# creating a sql database to store the contract abi-json

import sqlite3

conn = sqlite3.connect('contract_registry.db')
c = conn.cursor()

c.execute("CREATE TABLE IF NOT EXISTS contract_registry (address text, abi_json text )")
conn.commit()

In [44]:
#convert datatype of resistry dataframe to string

registry['abi-json'] = registry['abi-json'].astype(str)
registry['address'] = registry['address'].astype(str)

In [46]:
# inserting the contract abi-json into the database

for i in range(len(registry)):
    c.execute("INSERT INTO contract_registry (address, abi_json) VALUES (?,?)",(registry.iloc[i]['address'],registry.iloc[i]['abi-json']))
    conn.commit()

# fetching the contract abi-json from the database

c.execute("SELECT * FROM contract_registry")
print(c.fetchall())

[('0x0ef97d3e342b29c18bcbcb0426e3b5de4879c53c', '{\'status\': \'1\', \'message\': \'OK\', \'result\': \'[{"constant":true,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"_spender","type":"address"},{"name":"_value","type":"uint256"}],"name":"approve","outputs":[{"name":"success","type":"bool"}],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"_from","type":"address"},{"name":"_to","type":"address"},{"name":"_value","type":"uint256"}],"name":"transferFrom","outputs":[{"name":"success","type":"bool"}],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"standard","outputs":[{"name":"","type":"string"}],"payable":fal

In [53]:
c.execute("SELECT abi_json FROM contract_registry WHERE address = '0x0ef97d3e342b29c18bcbcb0426e3b5de4879c53c'")
print(c.fetchall())

[('{\'status\': \'1\', \'message\': \'OK\', \'result\': \'[{"constant":true,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"_spender","type":"address"},{"name":"_value","type":"uint256"}],"name":"approve","outputs":[{"name":"success","type":"bool"}],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"type":"function"},{"constant":false,"inputs":[{"name":"_from","type":"address"},{"name":"_to","type":"address"},{"name":"_value","type":"uint256"}],"name":"transferFrom","outputs":[{"name":"success","type":"bool"}],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"payable":false,"type":"function"},{"constant":true,"inputs":[],"name":"standard","outputs":[{"name":"","type":"string"}],"payable":false,"type":"function"},{"constant":true,"inputs